# 🚀 MINIMAL WORKING EMOTION DETECTION TRAINING
## Ultra-Simple Version That Should Work

**FEATURES:**
✅ Basic training (no complex arguments)
✅ Configuration preservation
✅ Simple data processing
✅ Model saving with verification

**Target**: Get training working first, then optimize

In [ ]:
# Install required packages
!pip install transformers torch scikit-learn numpy pandas

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
import json
import warnings
warnings.filterwarnings('ignore')

print('✅ All packages imported successfully')
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

## 🎯 SETUP

## 🔑 WANDB API KEY SETUP

In [ ]:
# Setup Weights & Biases API key from Google Colab secrets
import os
import wandb

print('🔑 SETTING UP WANDB API KEY')
print('=' * 40)

# Try to get API key from Colab secrets
try:
    from google.colab import userdata
    
    # Try different possible secret names
    possible_secret_names = [
        'WANDB_API_KEY',
        'wandb_api_key',
        'WANDB_KEY',
        'wandb_key',
        'WANDB_TOKEN',
        'wandb_token'
    ]
    
    api_key = None
    used_secret_name = None
    
    for secret_name in possible_secret_names:
        try:
            api_key = userdata.get(secret_name)
            used_secret_name = secret_name
            print(f'✅ Found API key in secret: {secret_name}')
            break
        except:
            continue
    
    if api_key:
        # Set the environment variable
        os.environ['WANDB_API_KEY'] = api_key
        print(f'✅ API key set from secret: {used_secret_name}')
        
        # Test wandb login
        try:
            wandb.login(key=api_key)
            print('✅ WandB login successful!')
        except Exception as e:
            print(f'⚠️ WandB login failed: {str(e)}')
            print('Continuing without WandB...')
    else:
        print('❌ No WandB API key found in secrets')
        print('\n📋 TO SET UP WANDB SECRET:')
        print('1. Go to Colab → Settings → Secrets')
        print('2. Add a new secret with name: WANDB_API_KEY')
        print('3. Value: Your WandB API key from https://wandb.ai/authorize')
        print('4. Restart runtime and run this cell again')
        print('\n⚠️ Continuing without WandB logging...')
        
except ImportError:
    print('⚠️ Google Colab secrets not available')
    print('\n📋 TO SET UP WANDB:')
    print('1. Get your API key from: https://wandb.ai/authorize')
    print('2. Run: wandb login')
    print('3. Enter your API key when prompted')
    print('\n⚠️ Continuing without WandB logging...')

print('\n✅ WandB setup completed')

In [ ]:
# Define emotion classes
emotions = ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
print(f'🎯 Emotion classes: {emotions}')

# Simple dataset
data = [
    {'text': 'I feel anxious about the presentation.', 'label': 0},
    {'text': 'I am feeling calm today.', 'label': 1},
    {'text': 'I feel content with my life.', 'label': 2},
    {'text': 'I am excited about the opportunity.', 'label': 3},
    {'text': 'I feel frustrated with the issues.', 'label': 4},
    {'text': 'I am grateful for the support.', 'label': 5},
    {'text': 'I feel happy about the success.', 'label': 6},
    {'text': 'I am hopeful for the future.', 'label': 7},
    {'text': 'I feel overwhelmed with tasks.', 'label': 8},
    {'text': 'I am proud of my achievements.', 'label': 9},
    {'text': 'I feel sad about the loss.', 'label': 10},
    {'text': 'I am tired from working.', 'label': 11},
    # Add more samples for each emotion
    {'text': 'I am worried about the results.', 'label': 0},
    {'text': 'I feel peaceful and relaxed.', 'label': 1},
    {'text': 'I am satisfied with the outcome.', 'label': 2},
    {'text': 'I feel thrilled about the news.', 'label': 3},
    {'text': 'I am annoyed with the problems.', 'label': 4},
    {'text': 'I feel thankful for the help.', 'label': 5},
    {'text': 'I am joyful about the completion.', 'label': 6},
    {'text': 'I feel optimistic about tomorrow.', 'label': 7},
    {'text': 'I am stressed with responsibilities.', 'label': 8},
    {'text': 'I feel accomplished and confident.', 'label': 9},
    {'text': 'I am depressed about the situation.', 'label': 10},
    {'text': 'I feel exhausted from the work.', 'label': 11}
]

print(f'📊 Dataset size: {len(data)} samples')

## 🔧 MODEL SETUP

In [ ]:
# Load model and tokenizer
model_name = 'j-hartmann/emotion-english-distilroberta-base'
print(f'🔧 Loading model: {model_name}')

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f'Original model labels: {AutoModelForSequenceClassification.from_pretrained(model_name).config.num_labels}')
print(f'Original id2label: {AutoModelForSequenceClassification.from_pretrained(model_name).config.id2label}')

# CRITICAL: Create a NEW model with correct configuration from scratch
print('\n🔧 CREATING NEW MODEL WITH CORRECT ARCHITECTURE')
print('=' * 60)

# Load the base model without the classification head
from transformers import RobertaModel
base_model = RobertaModel.from_pretrained(model_name)

# Create a new model with the correct number of labels
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(emotions),  # Set to 12 emotions
    ignore_mismatched_sizes=True  # Important: ignore size mismatches
)

# Configure the model properly
model.config.num_labels = len(emotions)
model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}
model.config.problem_type = 'single_label_classification'

# Verify the configuration
print(f'✅ Model created with {model.config.num_labels} labels')
print(f'✅ New id2label: {model.config.id2label}')
print(f'✅ Classifier output size: {model.classifier.out_proj.out_features}')
print(f'✅ Problem type: {model.config.problem_type}')

# Test the model with a sample input
test_input = tokenizer('I feel happy today', return_tensors='pt', truncation=True, padding=True)
with torch.no_grad():
    test_output = model(**test_input)
    print(f'✅ Test output shape: {test_output.logits.shape}')
    print(f'✅ Expected shape: [1, {len(emotions)}]')
    assert test_output.logits.shape[1] == len(emotions), f'Output shape mismatch: {test_output.logits.shape[1]} != {len(emotions)}'
    print('✅ Model architecture verified!')

# Move model to GPU
if torch.cuda.is_available():
    model = model.to('cuda')
    print('✅ Model moved to GPU')
else:
    print('⚠️ CUDA not available, model will run on CPU')

## 📝 DATA PREPROCESSING

In [ ]:
# Prepare data
texts = [item['text'] for item in data]
labels = [item['label'] for item in data]

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f'📊 Training samples: {len(train_texts)}')
print(f'📊 Validation samples: {len(val_texts)}')

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt')

# Create dataset class
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = SimpleDataset(train_encodings, train_labels)
val_dataset = SimpleDataset(val_encodings, val_labels)

print('✅ Data preprocessing completed')

## ⚙️ MINIMAL TRAINING ARGUMENTS

In [ ]:
# Minimal training arguments - only essential parameters
training_args = TrainingArguments(
    output_dir='./minimal_emotion_model',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    save_steps=50,
    eval_steps=50,
    # Disable wandb if no API key is set
    report_to=None if 'WANDB_API_KEY' not in os.environ else ['wandb']
)

print('✅ Minimal training arguments configured')
if 'WANDB_API_KEY' in os.environ:
    print('✅ WandB logging enabled')
else:
    print('⚠️ WandB logging disabled (no API key)')

## 📊 COMPUTE METRICS

In [ ]:
# Simple compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        'f1': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions)
    }

print('✅ Compute metrics function ready')

## 🚀 TRAINING

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print('✅ Trainer initialized')

# Start training
print('🚀 STARTING MINIMAL TRAINING')
print('=' * 40)
print(f'📊 Training samples: {len(train_dataset)}')
print(f'🧪 Validation samples: {len(val_dataset)}')

# Train the model
trainer.train()

print('✅ Training completed successfully!')

## 📈 EVALUATION

In [ ]:
# Evaluate the model
print('📈 EVALUATING MODEL')
print('=' * 40)

results = trainer.evaluate()
print('\n📊 FINAL RESULTS:')
print(f'F1 Score: {results["eval_f1"]:.4f}')
print(f'Accuracy: {results["eval_accuracy"]:.4f}')

print('✅ Evaluation completed!')

## 💾 MODEL SAVING

In [ ]:
# Save model
print('💾 SAVING MODEL')
print('=' * 30)

model_path = './minimal_emotion_model_final'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

print(f'✅ Model saved to: {model_path}')

# Verify configuration
config_path = f'{model_path}/config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

print(f'\n🔍 SAVED CONFIGURATION:')
print(f'Model type: {config.get("model_type", "NOT SET")}')
print(f'Number of labels: {config.get("num_labels", "NOT SET")}')
print(f'id2label: {config.get("id2label", "NOT SET")}')

print('\n✅ Model saving completed!')